**Упражнение 5** Рассмотрим нормальное распределение $\mathcal{N}(\theta,\sigma^2)$ с неизвестными параметрами $\theta\in\mathbb{R}$ и $\sigma^2>0$. В этой задаче мы численно сравним следующие доверительные интервалы для параметра $\theta$: 1) теоретический, 2) на основе параметрического бутстрэпа, 3) на основе непараметрического бустрэпа.

In [1]:
import numpy as np
from scipy import stats

In [2]:
np.random.seed(123)

#### (1) Сгенерируем неизвестный параметр $\theta$ из равномерного распределения на $[10,20]$ и $\sigma$ - из равномерного распределния $[1,2]$.

In [5]:
theta = np.random.uniform(10,20)
sigma = np.random.uniform(1,2)
print(theta, sigma)

12.861393349503794 1.226851453564203


#### (2) Сгенерируем выборку из нормального распределения $\mathcal{N}(\theta,\sigma^2)$ размера $n=50$.

In [18]:
alpha = 0.05
n = 50
sample = np.random.normal(theta, sigma, n)
sample

array([11.97661777, 10.77277248, 13.8031754 , 13.31616148, 15.40767368,
       13.01813306, 15.06959034, 14.38068491, 13.98243739, 13.43343496,
       13.01479844, 12.61890503, 11.96820265, 12.11262944, 11.79029195,
       13.05335761, 13.21295017, 13.92595419, 14.41644954, 15.44739989,
       12.57174666, 12.57666342, 11.53867645, 14.28844799, 13.50241106,
       15.12415549, 13.71474706, 12.63848769, 12.34334225, 14.10262404,
       12.77949192, 12.66992614, 14.80406936, 11.89635938, 13.48900158,
       14.62982672, 11.83463188, 13.42558799, 11.11104392, 12.17836865,
       13.08203017, 10.23045473, 11.96936317, 13.23854972, 12.38594086,
       11.87559111, 14.01339365, 13.3985328 , 14.26134321, 13.07173921])

#### (3) Построим доверительные интервалы 1), 2), 3) уровня доверия $1-\alpha$.

In [21]:
# теоретический доверительный интервал

s_est = (1/(n-1))*((sample-np.mean(sample))**2).sum() # несмещенная оценка дисперсии
# s_est_1 = np.var(sample, ddof=1)
# quantile = stats.t(df=n-1).ppf(alpha/2)
quant = stats.t(df=n-1).ppf(1-alpha/2) # квантиль уровня (1-aplha/2) распределения Стьюдента с (n-1) степенью свободы
# print(s_est, sigma**2, np.mean(sample))
# print(quantile_1, quantile_2)
ci_theoretical = [np.mean(sample)-quant*s_est/np.sqrt(n), np.mean(sample)+quant*s_est/np.sqrt(n)]
ci_theoretical

[12.706870693433915, 13.511856053571845]

In [27]:
# доверительный интервал на основе параметрического бутстрэпа

bs_number = 10 # количество бутстрэп-выборок
bs_size = 20 # рамзер бутстрэп-выборки
mean = np.mean(sample) # оценка параметра theta
dev = np.sqrt(np.var(sample, ddof=1)) # оценка параметра sigma
bs_samples = np.random.normal(mean, dev, size=[bs_number, bs_size]) # генерируем бутстрэп-выборки
bs_theta_est = np.apply_along_axis(np.mean, 1, bs_samples) # оцениваем theta по каждой выборке
ci_bs_par = [np.quantile(bs_theta_est, alpha/2), np.quantile(bs_theta_est,1-alpha/2)]
ci_bs_par

[12.830552573356997, 13.398812203567772]

In [34]:
# доверительный интервал на основе непараметрического бутстрэпа

bs_number = 10 # количество бутстрэп-выборок
bs_size = 20 # рамзер бутстрэп-выборки
bs_samples_np = np.random.choice(sample, size=[bs_number, bs_size]) # генерируем бутстрэп-выборки
bs_theta_est_np = np.apply_along_axis(np.mean, 1, bs_samples_np) # оцениваем theta по каждой выборке
ci_bs_np = [np.quantile(bs_theta_est_np, alpha/2), np.quantile(bs_theta_est_np, 1-alpha/2)]
ci_bs_np

[12.663347483141209, 13.440020377313827]

#### (4) Сравним длины полученных интервалов

In [35]:
print('Длина теоретического доверительного интервала:', ci_theoretical[1] - ci_theoretical[0])
print('Длина интервала на основе параметрического бутстрэпа:', ci_bs_par[1] - ci_bs_par[0])
print('Длина интервала на основе непараметрического бутстрэпа:', ci_bs_np[1] - ci_bs_np[0])

Длина теоретического доверительного интервала: 0.8049853601379304
Длина интервала на основе параметрического бутстрэпа: 0.5682596302107754
Длина интервала на основе непараметрического бутстрэпа: 0.7766728941726182


#### (5) Проведите этот эксперимент на 10 000 выборок. С какой частотой $\theta$ попадает в полученные доверительные интервалы?

In [41]:
samples_num = 10000
theor = np.zeros(samples_num)
bs_par = np.zeros(samples_num)
bs_np = np.zeros(samples_num)

In [42]:
def CI_theor(sample, alpha):
    n = len(sample)
    s_est = np.var(sample, ddof=1)
    quant = stats.t(df=n-1).ppf(1-alpha/2)
    return [np.mean(sample)-quant*s_est/np.sqrt(n), np.mean(sample)+quant*s_est/np.sqrt(n)]

In [43]:
def CI_bs_par(sample, alpha, bs_num, bs_size):
    mean = np.mean(sample)
    dev = np.sqrt(np.var(sample, ddof=1))
    bs_samples = np.random.normal(mean, dev, size=[bs_num, bs_size])
    bs_theta_est = np.apply_along_axis(np.mean, 1, bs_samples)
    return [np.quantile(bs_theta_est, alpha/2), np.quantile(bs_theta_est,1-alpha/2)]

In [46]:
def CI_bs_np(sample, alpha, bs_num, bs_size):
    bs_samples_np = np.random.choice(sample, size=[bs_num, bs_size])
    bs_theta_est_np = np.apply_along_axis(np.mean, 1, bs_samples_np)
    return [np.quantile(bs_theta_est_np, alpha/2), np.quantile(bs_theta_est_np, 1-alpha/2)]

In [47]:
alpha = 0.05
number_of_bs_samples = 10
size_of_bs_samples = 20

for i in range(samples_num):
    sample = np.random.normal(theta, sigma, size=n)
    
    ci_theoretical = CI_theor(sample, alpha)
    theor[i] = ((theta >= ci_theoretical[0]) and (theta <= ci_theoretical[1]))
    
    ci_bs_par = CI_bs_par(sample, alpha, number_of_bs_samples, size_of_bs_samples)
    bs_par[i] = ((theta >= ci_bs_par[0]) and (theta <= ci_bs_par[1]))
    
    ci_bs_np = CI_bs_np(sample, alpha, number_of_bs_samples, size_of_bs_samples)
    bs_np[i] = ((theta >= ci_bs_np[0]) and (theta <= ci_bs_np[1]))

In [48]:
print("Частота попадания истинного параметра в доверительный интервал:")
print("- для теоретического доверительного интервала ", np.mean(theor))
print("- для параметрического бутстрэп доверительного интервала ", np.mean(bs_par))
print("- для непараметрического бутстрэп доверительного интервала ", np.mean(bs_np))

Частота попадания истинного параметра в доверительный интервал:
- для теоретического доверительного интервала  0.9746
- для параметрического бутстрэп доверительного интервала  0.9172
- для непараметрического бутстрэп доверительного интервала  0.9143


Как мы видим, истинное значение параметра $\theta$ чаще попадает в теоретический доверительный интервал, чем в доверительные интервалы на основе бутстрэпа.

**Упражнение 6** В файле Banner_small.npy даны две выборки из распределения Бернулли. Они соответствуют кликам по двум рекламным баннерам в Интернете. Постройте доверительные интервалы для частоты кликов (то есть параметра "успеха"). Какой из изученный вариантов доверительных интервалов лучше применять в этой задаче? Как думаете, можно ли сказать, что один из баннеров лучше, чем другой? Если да, то с какой вероятностью? Проверьте свой вывод на выборке большего размера, которая записана в файл Banner_large.npy

In [49]:
import numpy as np
import statsmodels.stats.proportion as st

In [50]:
np.random.seed(123)

In [59]:
data_small = np.load('Banner_small.npy')

In [81]:
banner_a = data_small[0]
banner_b = data_small[1]
print(sum(banner_a)/len(banner_a), sum(banner_b)/len(banner_b))

0.002 0.006


In [60]:
alpha = 0.05

In [73]:
# Асимптотический доверительный интервал на основе ЦПТ

ci_normal_a = st.proportion_confint(sum(banner_a), banner_a.size, alpha, method = 'normal')
ci_normal_b = st.proportion_confint(sum(banner_b), banner_b.size, alpha, method = 'normal')
print('Асимптотический доверительный интервал на основе ЦПТ равен:')
print('   - для баннера "a" ', ci_normal_a)
print('   - для баннера "b" ', ci_normal_b)

Асимптотический доверительный интервал на основе ЦПТ равен:
   - для баннера "a"  (0.0, 0.005916006079184627)
   - для баннера "b"  (0.0, 0.012769115216425226)


In [74]:
# Доверительный интервал Уилсона

ci_wilson_a = st.proportion_confint(sum(banner_a), banner_a.size, alpha, method = 'wilson')
ci_wilson_b = st.proportion_confint(sum(banner_b), banner_b.size, alpha, method = 'wilson')
print('Доверительный интервал Уилсона равен:')
print('   - для баннера "a" ', ci_wilson_a)
print('   - для баннера "b" ', ci_wilson_b)

Доверительный интервал Уилсона равен:
   - для баннера "a"  (0.00035313639455927543, 0.01124070670514676)
   - для баннера "b"  (0.002042596271960238, 0.017490252104053382)


В случае с относительно небольшим количеством наблюдений (в данном случае их по 500 на каждый баннер) и, к тому же, при условии, что значение вероятности успеха близко к нулю, предпочтительным является использование доверительного интеравала Уилсона. Он более устойчив к подобным ситуациям, и позволяет получить более надежные (близкие к истинному значению параметра) оценки.

При помощи арифметического среднего можно оценить вероятность успеха для баннера "а" в 0.002, а вероятность успеха для баннера "b" - в 0.006. Как видно, вероятность успеха для баннера "b" выше. Но с учетом того, что размеры выборок относительно небольшие и что сами значения вероятностей (оценок) очень близки к нулю, говорить с какой-то уверенностью о том, что баннер "b" лучше баннера "a" все же сложно.

Проверим наши выводы на выборках большего размера

In [75]:
data_large = np.load('Banner_large.npy')

In [82]:
a_large = data_large[0]
b_large = data_large[1]
print(sum(a_large)/len(a_large), sum(b_large)/len(b_large))

0.00547 0.00645


In [85]:
# Асимптотический доверительный интервал на основе ЦПТ

ci_normal_a_l = st.proportion_confint(sum(a_large), a_large.size, alpha, method = 'normal')
ci_normal_b_l = st.proportion_confint(sum(b_large), b_large.size, alpha, method = 'normal')
print('Асимптотический доверительный интервал на основе ЦПТ равен:')
print('   - для баннера "a" ', ci_normal_a_l)
print('   - для баннера "b" ', ci_normal_b_l)

Асимптотический доверительный интервал на основе ЦПТ равен:
   - для баннера "a"  (0.005012858448137027, 0.005927141551862973)
   - для баннера "b"  (0.005953838785243252, 0.006946161214756748)


In [86]:
# Доверительный интервал Уилсона

ci_wilson_a_l = st.proportion_confint(sum(a_large), a_large.size, alpha, method = 'wilson')
ci_wilson_b_l = st.proportion_confint(sum(b_large), b_large.size, alpha, method = 'wilson')
print('Доверительный интервал Уилсона равен:')
print('   - для баннера "a" ', ci_wilson_a_l)
print('   - для баннера "b" ', ci_wilson_b_l)

Доверительный интервал Уилсона равен:
   - для баннера "a"  (0.00502901983248579, 0.005948973040645592)
   - для баннера "b"  (0.00597018712933124, 0.00696773045409948)


Как мы видим, в случае с выборками существенно большего размера оба доверительных интервала дают очень похожий результат. При этом видно, что оценка параметра для баннера "b" лежит в интервале, сдвинутом вправо относительно интервала для баннера "a" почти на 0.001. Это говорит о том, что вероятность успеха в случае с баннером "b" скорее всего выше, чем в случае с баннером "a".